In [1]:
import os
%load_ext dotenv
%dotenv
from langchain.llms import OpenAI

In [3]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

In [4]:
fromat_instruction = output_parser.get_format_instructions()

In [5]:
fromat_instruction

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [6]:
response = "one, two, three"
output_parser.parse(response)

['one', 'two', 'three']

In [7]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate
)
from langchain.schema import (
AIMessage,
HumanMessage,
SystemMessage
)

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

In [9]:
human_template = "{request}\n {format_instructions}"
human_prompt_template = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_prompt_template])

In [10]:
chat_prompt.input_variables

['format_instructions', 'request']

In [11]:
request = chat_prompt.format_prompt(request = "Give me 5 charactersitcis of a dog",
                         format_instructions = output_parser.get_format_instructions()).to_messages()

In [12]:
result = llm(request)

In [13]:
result

AIMessage(content='Loyal, friendly, playful, protective, obedient', additional_kwargs={}, example=False)

In [14]:
output_parser.parse(result.content)

['Loyal', 'friendly', 'playful', 'protective', 'obedient']

In [15]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [16]:
class Scientist(BaseModel):
    name: str = Field(description="Name of Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [18]:
parser = PydanticOutputParser(pydantic_object= Scientist)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "Name of Scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "Python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}
```


In [30]:
template_text = "{request} \n {format_instruction}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])


In [27]:
req = chat_prompt.format_prompt(request = "Tell me discoveries of a famous scientist", format_instruction = parser.get_format_instructions()).to_messages()

In [33]:
result = llm(req)

In [34]:
parser.parse(result.content)

Scientist(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Brownian Motion'])